In [1]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

In [2]:
session_key_list = pd.read_csv('session_keys.csv')

In [3]:

for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/stints?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_stints = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_stints = pd.concat([df_stints,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')

Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.
Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 ext

In [4]:
df_stints

,meeting_key,session_key,stint_number,driver_number,lap_start,lap_end,compound,tyre_age_at_start
0,1141,7953,1,2,1.0,12.0,SOFT,0
1,1141,7953,1,22,1.0,10.0,SOFT,0
2,1141,7953,1,23,1.0,11.0,SOFT,0
3,1141,7953,1,63,1.0,13.0,SOFT,3
4,1141,7953,1,18,1.0,15.0,SOFT,3
...,...,...,...,...,...,...,...,...
56,1275,9850,3,30,33.0,57.0,HARD,0
57,1275,9850,3,22,33.0,57.0,HARD,0
58,1275,9850,3,23,33.0,57.0,MEDIUM,0
59,1275,9850,3,44,33.0,57.0,MEDIUM,0


In [5]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [6]:
dtype_dict = {
    'meeting_key': types.Integer,
    'session_key': types.Integer,
    'stint_number': types.Integer,
    'driver_number': types.Integer,
    'lap_start': types.Float,      # values like 1.0, 33.0 → float
    'lap_end': types.Float,        # same reason
    'compound': types.String,      # SOFT / MEDIUM / HARD
    'tyre_age_at_start': types.Integer
}

In [7]:

df_stints.to_sql(name = 'raw_stints', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

892